In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
train_features = pd.read_csv("/kaggle/input/lish-moa/train_features.csv")

In [ ]:
train_features.head()

In [ ]:
train_targets_scored = pd.read_csv("/kaggle/input/lish-moa/train_targets_scored.csv")

All of the ctl_vehicle rows don't have a a MOA because there was no treatment (mentioned in data description). I am going to use all of these for training (should have also taken the ones from the test data), because I can use them for normalization of the treatments in the different dose and time conditions. 

I calculated 50 PCAs for the scaled genes and cell viabiability values. Then the mean vectors of PC values for the ctl_vehicles in each dose+time condition was subtracted from the treatments then those were all converted to unit vectors so magnitudes of changes would be less important than the direction of change. Then these 100 PCA dimensions of each treatment condition (50 gene + gene cell viability) were used to train a fully conected neural network 

In [ ]:
train_features.shape

In [ ]:
train_features['cp_type'].value_counts()

In [ ]:
train_features['sig_id'].value_counts() # all different IDs

In [ ]:
train_targets_scored[train_features['cp_type']=="ctl_vehicle"].sum(axis=1).value_counts()

In [ ]:
ctl_train_features = train_features[train_features['cp_type']=="ctl_vehicle"]

In [ ]:
train_targets_scored = train_targets_scored[train_features['cp_type'] != "ctl_vehicle"]

In [ ]:
train_features = train_features[train_features['cp_type'] != "ctl_vehicle"]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_features, train_targets_scored, test_size=0.20, random_state=11)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
X_train_with_ctl = pd.concat([X_train,ctl_train_features],axis=0)

In [ ]:
X_train.shape

In [ ]:
X_train_with_ctl.shape

In [ ]:
scaler = StandardScaler()
scaler = scaler.fit(X_train_with_ctl.iloc[:,4:])
X_train_scaled = scaler.transform(X_train_with_ctl.iloc[:,4:])

In [ ]:
pca_genes = PCA(n_components=50)
pca_genes.fit(X_train_scaled[:,0:-100])

In [ ]:
pca_genes.explained_variance_ratio_

In [ ]:
plt.plot(np.cumsum(pca_genes.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

In [ ]:
plt.plot(np.arange(0,50), pca_genes.explained_variance_ratio_)
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Proportion of Variance Explained');

In [ ]:
pca_genes_matrix = pca_genes.transform(X_train_scaled[:,0:-100])

In [ ]:
pca_genes_matrix.shape

In [ ]:
pca_viability = PCA(n_components=50)
pca_viability.fit(X_train_scaled[:,-100:])

In [ ]:
pca_viability.explained_variance_ratio_

In [ ]:
plt.plot(np.cumsum(pca_viability.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

In [ ]:
plt.plot(np.arange(0,50), pca_viability.explained_variance_ratio_)
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Proportion of Variance Explained');

In [ ]:
pca_viability_matrix = pca_viability.transform(X_train_scaled[:,-100:])

In [ ]:
X_train_with_ctl[X_train_with_ctl["cp_type"]=="ctl_vehicle"]['cp_dose'].value_counts()

In [ ]:
X_train_with_ctl[X_train_with_ctl["cp_type"]=="ctl_vehicle"]['cp_time'].value_counts()

In [ ]:
X_train_with_ctl[X_train_with_ctl["cp_type"]!="ctl_vehicle"]['cp_dose'].value_counts()

In [ ]:
X_train_with_ctl[X_train_with_ctl["cp_type"]!="ctl_vehicle"]['cp_time'].value_counts()

In [ ]:
X_train_with_ctl.reset_index(inplace=True)

In [ ]:
D1_48 = X_train_with_ctl[(X_train_with_ctl["cp_type"]=="ctl_vehicle") &\
                                  (X_train_with_ctl["cp_time"]==48) &\
                                  (X_train_with_ctl['cp_dose']=='D1')].index

In [ ]:
D1_72 = X_train_with_ctl[(X_train_with_ctl["cp_type"]=="ctl_vehicle") &\
                                  (X_train_with_ctl["cp_time"]==72) &\
                                  (X_train_with_ctl['cp_dose']=='D1')].index

In [ ]:
D1_24 = X_train_with_ctl[(X_train_with_ctl["cp_type"]=="ctl_vehicle") &\
                                  (X_train_with_ctl["cp_time"]==24) &\
                                  (X_train_with_ctl['cp_dose']=='D1')].index

In [ ]:
D2_72 = X_train_with_ctl[(X_train_with_ctl["cp_type"]=="ctl_vehicle") &\
                                  (X_train_with_ctl["cp_time"]==72) &\
                                  (X_train_with_ctl['cp_dose']=='D2')].index

In [ ]:
D2_48 = X_train_with_ctl[(X_train_with_ctl["cp_type"]=="ctl_vehicle") &\
                                  (X_train_with_ctl["cp_time"]==48) &\
                                  (X_train_with_ctl['cp_dose']=='D2')].index

In [ ]:
D2_24 = X_train_with_ctl[(X_train_with_ctl["cp_type"]=="ctl_vehicle") &\
                                  (X_train_with_ctl["cp_time"]==24) &\
                                  (X_train_with_ctl['cp_dose']=='D2')].index

In [ ]:
D1_72_gene_pc = pca_genes_matrix[D1_72].mean(axis=0)
D1_48_gene_pc = pca_genes_matrix[D1_48].mean(axis=0)
D1_24_gene_pc = pca_genes_matrix[D1_24].mean(axis=0)
D2_72_gene_pc = pca_genes_matrix[D2_72].mean(axis=0)
D2_48_gene_pc = pca_genes_matrix[D2_48].mean(axis=0)
D2_24_gene_pc = pca_genes_matrix[D2_24].mean(axis=0)

D1_72_viability_pc = pca_viability_matrix[D1_72].mean(axis=0)
D1_48_viability_pc = pca_viability_matrix[D1_48].mean(axis=0)
D1_24_viability_pc = pca_viability_matrix[D1_24].mean(axis=0)
D2_72_viability_pc = pca_viability_matrix[D2_72].mean(axis=0)
D2_48_viability_pc = pca_viability_matrix[D2_48].mean(axis=0)
D2_24_viability_pc = pca_viability_matrix[D2_24].mean(axis=0)

In [ ]:
def get_final_pcas(dose,time,control_gene_pc,control_viability_pc):
    indices = X_train_with_ctl[(X_train_with_ctl["cp_type"]!="ctl_vehicle") &\
                                  (X_train_with_ctl["cp_time"]==time) &\
                                  (X_train_with_ctl['cp_dose']==dose)].index
    final_gene_pcas=[]
    final_viability_pcas=[]
    for i in pca_genes_matrix[indices]:
        corrected_pca = i - control_gene_pc
        final_gene_pcas.append(corrected_pca/np.linalg.norm(corrected_pca))
    final_gene_pcas=pd.DataFrame(final_gene_pcas)
    for i in pca_viability_matrix[indices]:
        corrected_pca = i - control_viability_pc
        final_viability_pcas.append(corrected_pca/np.linalg.norm(corrected_pca))
    final_viability_pcas=pd.DataFrame(final_viability_pcas)
    final_pcas=pd.concat([final_gene_pcas,final_viability_pcas],axis=1)
    final_pcas['index']=indices
    
    return final_pcas
    

In [ ]:
D1_72_final_pcas=get_final_pcas('D1',72,D1_72_gene_pc,D1_72_viability_pc)
D1_48_final_pcas=get_final_pcas('D1',48,D1_48_gene_pc,D1_48_viability_pc)
D1_24_final_pcas=get_final_pcas('D1',24,D1_24_gene_pc,D1_24_viability_pc)
D2_72_final_pcas=get_final_pcas('D2',72,D2_72_gene_pc,D2_72_viability_pc)
D2_48_final_pcas=get_final_pcas('D2',48,D2_48_gene_pc,D2_48_viability_pc)
D2_24_final_pcas=get_final_pcas('D2',24,D2_24_gene_pc,D2_24_viability_pc)

In [ ]:
final_pcas = pd.concat([D1_72_final_pcas,D1_48_final_pcas,D1_24_final_pcas,D2_72_final_pcas,D2_48_final_pcas,D2_24_final_pcas],axis=0)

In [ ]:
final_pcas.shape

In [ ]:
X_train_with_ctl.head(8)

In [ ]:
X_train_with_ctl.index

In [ ]:
X_train_final = pd.merge(final_pcas,X_train_with_ctl,how="left",left_on='index',right_index=True)

In [ ]:
X_train_final.shape

In [ ]:
y_train.shape

In [ ]:
X_train_final = X_train_final.iloc[:,0:103]

In [ ]:
X_train_final.drop(['index_x','index_y'],inplace=True,axis=1)

In [ ]:
X_train_y_final = pd.merge(X_train_final,y_train,how="inner",left_on="sig_id",right_on="sig_id")

In [ ]:
X_train_y_final.iloc[:,0:103].head()

In [ ]:
X_train_final = X_train_y_final.iloc[:,0:100]

In [ ]:
y_train_final = X_train_y_final.iloc[:,101:]

In [ ]:
y_train_final.head()

In [ ]:
X_test_scaled = scaler.transform(X_test.iloc[:,4:])

In [ ]:
X_test_pca_viability = pca_viability.transform(X_test_scaled[:,-100:])

In [ ]:
X_test_pca_gene = pca_genes.transform(X_test_scaled[:,:-100])

In [ ]:
X_test.reset_index(inplace=True)

In [ ]:
X_test.head()

In [ ]:
def get_final_pcas_test(dose,time,control_gene_pc,control_viability_pc):
    indices = X_test[(X_test["cp_type"]!="ctl_vehicle") & (X_test["cp_time"]==time) &\
                                  (X_test['cp_dose']==dose)].index
    
    final_gene_pcas=[]
    final_viability_pcas=[]
    for i in X_test_pca_gene[indices]:
        corrected_pca = i - control_gene_pc
        final_gene_pcas.append(corrected_pca/np.linalg.norm(corrected_pca))
    final_gene_pcas=pd.DataFrame(final_gene_pcas)
    for i in X_test_pca_viability[indices]:
        corrected_pca = i - control_viability_pc
        final_viability_pcas.append(corrected_pca/np.linalg.norm(corrected_pca))
    final_viability_pcas=pd.DataFrame(final_viability_pcas)
    final_pcas=pd.concat([final_gene_pcas,final_viability_pcas],axis=1)
    final_pcas['index']=indices
    
    return final_pcas
    

In [ ]:
D1_72_final_pcas_test=get_final_pcas_test('D1',72,D1_72_gene_pc,D1_72_viability_pc)
D1_48_final_pcas_test=get_final_pcas_test('D1',48,D1_48_gene_pc,D1_48_viability_pc)
D1_24_final_pcas_test=get_final_pcas_test('D1',24,D1_24_gene_pc,D1_24_viability_pc)
D2_72_final_pcas_test=get_final_pcas_test('D2',72,D2_72_gene_pc,D2_72_viability_pc)
D2_48_final_pcas_test=get_final_pcas_test('D2',48,D2_48_gene_pc,D2_48_viability_pc)
D2_24_final_pcas_test=get_final_pcas_test('D2',24,D2_24_gene_pc,D2_24_viability_pc)

In [ ]:
final_pcas_test = pd.concat([D1_72_final_pcas_test,D1_48_final_pcas_test,D1_24_final_pcas_test,D2_72_final_pcas_test,D2_48_final_pcas_test,D2_24_final_pcas_test],axis=0)

In [ ]:
X_test_final = pd.merge(final_pcas_test,X_test,how="left",left_on='index',right_index=True)

In [ ]:
X_test_final.drop(['index_x','index_y'],inplace=True,axis=1)

In [ ]:
X_y_test_final = pd.merge(X_test_final,y_test,how="inner",left_on="sig_id",right_on="sig_id")

In [ ]:
X_test_final = X_y_test_final.iloc[:,0:100]

In [ ]:
y_test_final = X_y_test_final.iloc[:,101:]

In [ ]:
test_treat_ids = X_y_test_final.iloc[100]

In [ ]:
y_test_final = y_test_final.iloc[:,-206:]

In [ ]:
X_train_final.shape

In [ ]:
X_test_final.shape

In [ ]:
weights = dict(y_train_final.sum(axis=0))

In [ ]:
for i, k in zip(range(0,206),list(weights.keys())):
    weights[i]=weights[k]/y_train_final.sum(axis=0).min()
    del weights[k]

In [ ]:
weights

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

In [ ]:
model=Sequential()
model.add(Dense(units=100,
               activation="relu",
               use_bias=True,
               kernel_initializer="glorot_uniform"))
model.add(Dropout(0.1))
model.add(Dense(units=50,
               activation="relu",
               use_bias=True,
               kernel_initializer="glorot_uniform"))
model.add(Dropout(0.1))
model.add(Dense(units=100,
               activation="relu",
               use_bias=True,
               kernel_initializer="glorot_uniform"))
model.add(Dropout(0.1))
model.add(Dense(units=100,
               activation="relu",
               use_bias=True,
               kernel_initializer="glorot_uniform"))
model.add(Dropout(0.1))

model.add(Dense(units=206))
model.add(Activation('sigmoid')) #can I use softmax if there are some in multiple classes or no classes?

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
callback = EarlyStopping(monitor='loss', patience=10)

In [ ]:
from keras.optimizers import Adam

In [ ]:
optimizer = Adam(lr=0.000001) #model only seems to converge with very low learning rate

In [ ]:
from keras.losses import KLDivergence

In [ ]:
model.compile(loss=KLDivergence(), #is this appropriate for a classification problem where there are some falling in multiple classes?
              optimizer=optimizer, 
              metrics=['accuracy'])

In [ ]:
#note.. how to avoid webpage reloading when using significant energy?

model.fit(X_train_final, y_train_final, epochs=300, batch_size=3,
              validation_data=(X_test_final, y_test_final),class_weight=weights,
         verbose=True, callbacks=[callback])

In [ ]:
test_features = pd.read_csv("/kaggle/input/lish-moa/test_features.csv")

In [ ]:
test_features.head()

In [ ]:
test_scaled = scaler.transform(test_features.iloc[:,4:])

In [ ]:
test_pca_viability = pca_viability.transform(test_scaled[:,-100:])

In [ ]:
test_pca_gene = pca_genes.transform(test_scaled[:,0:-100])

In [ ]:
test_features.reset_index(inplace=True)

In [ ]:
def get_final_pcas_final_test(dose,time,control_gene_pc,control_viability_pc):
    indices = test_features[(test_features["cp_type"]!="ctl_vehicle") & (test_features["cp_time"]==time) &\
                                  (test_features['cp_dose']==dose)].index
    
    final_gene_pcas=[]
    final_viability_pcas=[]
    for i in test_pca_gene[indices]:
        corrected_pca = i - control_gene_pc
        final_gene_pcas.append(corrected_pca/np.linalg.norm(corrected_pca))
    final_gene_pcas=pd.DataFrame(final_gene_pcas)
    for i in test_pca_viability[indices]:
        corrected_pca = i - control_viability_pc
        final_viability_pcas.append(corrected_pca/np.linalg.norm(corrected_pca))
    final_viability_pcas=pd.DataFrame(final_viability_pcas)
    final_pcas=pd.concat([final_gene_pcas,final_viability_pcas],axis=1)
    final_pcas['index']=indices
    
    return final_pcas
    

In [ ]:
D1_72_final_pcas_val=get_final_pcas_final_test('D1',72,D1_72_gene_pc,D1_72_viability_pc)
D1_48_final_pcas_val=get_final_pcas_final_test('D1',48,D1_48_gene_pc,D1_48_viability_pc)
D1_24_final_pcas_val=get_final_pcas_final_test('D1',24,D1_24_gene_pc,D1_24_viability_pc)
D2_72_final_pcas_val=get_final_pcas_final_test('D2',72,D2_72_gene_pc,D2_72_viability_pc)
D2_48_final_pcas_val=get_final_pcas_final_test('D2',48,D2_48_gene_pc,D2_48_viability_pc)
D2_24_final_pcas_val=get_final_pcas_final_test('D2',24,D2_24_gene_pc,D2_24_viability_pc)

In [ ]:
final_pcas_test = pd.concat([D1_72_final_pcas_val,D1_48_final_pcas_val,D1_24_final_pcas_val,D2_72_final_pcas_val,D2_48_final_pcas_val,D2_24_final_pcas_val],axis=0)

In [ ]:
test_final = pd.merge(final_pcas_test,test_features,how="left",left_on='index',right_index=True)

In [ ]:
test_final = test_final.iloc[:,0:100]

In [ ]:
test_final.shape

In [ ]:
preds = model.predict(test_final)

In [ ]:
submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')

In [ ]:
x = pd.merge(final_pcas_test,test_features,how="left",left_on='index',right_index=True)

In [ ]:
ids = x.iloc[:,102]

In [ ]:
preds = pd.DataFrame(preds)

In [ ]:
preds.columns = y_train_final.columns

In [ ]:
preds.sum(axis=1).sort_values()

In [ ]:
preds.shape

In [ ]:
len(ids)

In [ ]:
preds.reset_index(inplace=True)

In [ ]:
preds = preds.reindex(columns=submission.columns)

In [ ]:
ids=list(ids)

In [ ]:
preds["sig_id"]=ids

In [ ]:
preds.head()

In [ ]:
preds.iloc[:,1:].apply(np.argmax,axis=1).value_counts()

In [ ]:
preds.iloc[:,1:].apply(np.max,axis=1).sort_values()

In [ ]:
test_features.iloc[:,1]

In [ ]:
submission2 = pd.merge(test_features.iloc[:,1],preds,how="left",left_on="sig_id",right_on="sig_id")

In [ ]:
submission2.head()

In [ ]:
submission = submission2.fillna(0.0)

In [ ]:
submission.to_csv('submission.csv')

In [ ]:
submission.drop(['sig_id'],axis=1).apply(lambda x: x > 0.25, axis=0).sum(axis=0).sort_values()